In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
load_dotenv()
from langchain.callbacks import StdOutCallbackHandler



import os
llm=OpenAI(temperature=0.2)


vectordb_file_path = "vectordb"
instructor_embeddings = OpenAIEmbeddings()
def load_data():
    loader = CSVLoader(file_path="data_2.csv",source_column="Sentence")
    documents = loader.load()
    # Create a FAISS instance for vector database from 'data'
    vectordb = FAISS.from_documents(documents=documents,
                                    embedding=instructor_embeddings)

    # Save vector database locally
    vectordb.save_local(vectordb_file_path)

def get_response():
    vectordb = FAISS.load_local(vectordb_file_path, instructor_embeddings)
    retriever = vectordb.as_retriever(score_threshold=0.7,k=4)

    prompt_template = """Given the following context and a question, generate an answer based on this context only.
    In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
    If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

    CONTEXT: {context}

    QUESTION: {question}"""

    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )


    response = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", 
                                         input_key="query",
                                         retriever=retriever, return_source_documents=True,
                                         chain_type_kwargs={"prompt": PROMPT},
                                         callbacks=[StdOutCallbackHandler()],
                                         verbose=True,
                                         )

    return response

if __name__=="__main__":
    load_data() 
    chain = get_response()
    print(chain("Do you any offers in Zara within a week expiring?"))